### กลุ่ม Diabetes Prediction 
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1 

### วัตถุประสงค์ของระบบต้นแบบ
กลุ่มของพวกเราเล็งเห็นถึงความสำคัญทางด้านสุขภาพของประชาชนคนไทย ในปัจจุบันคนไทยประสบกับปัญหาสุขภาพมากมายแต่หนึ่งปัญหาที่สร้างความเสียหายให้กับสุขภาพและคนไทยเป็นกันมากที่สุดนั่นคือโรคเบาหวาน พฤติกรรมการบริโภคของคนไทยในสมัยนี้ก็เป็นปัจจัยหนึ่งที่ทำให้ปริมาณผู้ป่วยโรคเบาหวานเพิ่มมากยิ่งขึ้น พวกเราจึงอยากทำระบบต้นแบบที่ช่วยแบ่งเบาภาระของเจ้าหน้าที่ในหน่วยงานสาธารณะสุข หมอ พยาบาล ในการวินิจฉัยโรคเบาหวาน ระบบต้นแบบอาจจะไม่สามารถชี้ชัดได้ 100 เปอเซนแต่สามารถคัดกรองผู้ป่วยและแบ่งเบาภาระได้ส่วนหนึ่งซึ่งพวกเราคิดว่ามันจะเป็นประโยชน์อย่างมากในการคัดกรองผู้ป่วย

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ
Link to data: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

In [1]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### ปั​ญห​า ML ของตั​วเอ​งเป็นปั​ญหา​ป​ระเภ​ทใด (classificati​on/regres​sion/clust​ering/ge​neration)
ปัญหา ML ของกลุ่มพวกเราเป็นปัญหาแบบ classification binary เนื่องจากเป็นการทำนายว่าข้อมูลของบุคคลที่นำเข้ามาในโมเดลนั้นเป็นหรือไม่เป็นโรคเบาหวานจึงเป็นการ classification binary อย่างชัดเจน

### f​eat​ur​es ที่ใช้ในการทำนา​ยมีอะ​ไรบ้าง
ก่อนที่เราจะสามารถดูค่าความสำคัญของแต่ละฟีเจอร์ได้นั้น เราต้องทำ Feature Engineer กับข้อมูลให้เรียบร้อยก่อนโดยมีขั้นตอนดังต่อไปนี้

แยกชุดข้อมูลออกเป็น ชุดข้อมูลเรียนรู้(train) ชุดข้อมูลทดสอบ(test) และชุดข้อมูลตรวจสอบความแม่นยำของโมเดล(validation)

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, stratify=diabetes["Diabetes_binary"]
                               , test_size=0.3, random_state=1234)
test, validation = train_test_split(test, stratify=test["Diabetes_binary"],
                                    test_size=0.3, random_state=1234)
len(train), len(test), len(validation)

(177576, 53272, 22832)

เนื่องจากข้อมูลของกลุ่มเรานั้น ขาดความสมดุลอย่างมากสังเกตได้จาก จำนวนของ Label ที่มีจำนวนไม่เท่ากัน

In [3]:
diabetes.Diabetes_binary.value_counts()

Diabetes_binary
0.0    218334
1.0     35346
Name: count, dtype: int64

ดังนั้นต้องทำให้ชุดข้อมูลที่นำมาใช้งานนั้นมีความสมดุลกันก่อนด้วยการทำ Under Sampling

In [4]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(random_state=1234)

feature = diabetes.drop(columns="Diabetes_binary").columns
label = "Diabetes_binary"

train[feature], train[label] = resampler.fit_resample(train[feature], train[label])
train.dropna(inplace=True)

train.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
186497,1.0,1.0,0.0,1.0,29.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,5.0,2.0,6.0,1.0,1.0,13.0,3.0,2.0
18001,0.0,0.0,0.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,6.0,8.0
37591,0.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,8.0,6.0,8.0
119797,0.0,1.0,1.0,1.0,30.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,4.0,7.0
55137,1.0,1.0,0.0,1.0,37.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,15.0,20.0,1.0,0.0,5.0,5.0,5.0


ขั้นตอนต่อมาคือการทำ Pipeline ของ Logistic Regression และ Decision Tree โดยเหตุผลที่เลือกทำ pipeline ทั้ง 2 โมเดลเพราะ ต้องการหาโมเดลที่มีความแม่นยำสูงและไม่ Overfit จนเกินไป

สร้าง Pipeline ของ Logistic Regression

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

logistic_pipeline = create_pipline_logistic()
logistic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

สร้างโมเดล Logistic Regression และตรวจสอบความแม่นยำ

In [6]:
logistic_pipeline.fit(train[feature], train[label])
logistic_pipeline.score(validation[feature], validation[label])

0.736291170287316

สร้าง Pipeline ของ SDG Classifier

In [12]:
from sklearn.linear_model import SGDClassifier

def create_pipline_sgd():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        SGDClassifier()
    )

sgd_pipline = create_pipline_sgd()
sgd_pipline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('sgdclassifier', SGDClassifier())])

สร้างโมเดล SDG Classifier และตรวจสอบความแม่นยำ

In [ ]:
sgd_pipline.fit(train[feature], train[label])
sgd_pipline.score(train[feature], train[label])

สร้าง Pipeline ของ Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

def create_pipline_decisiontree():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        DecisionTreeClassifier()
    )

decision_pipeline = create_pipline_decisiontree()
decision_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

สร้างโมเดล Decision Tree และตรวจสอบความแม่นยำ

In [8]:
decision_pipeline.fit(train[feature], train[label])
decision_pipeline.score(validation[feature], validation[label])

0.6750613174491941

ตรวจสอบค่าความสำคัญของแต่ละฟีเจอร์จากโมเดล Logistic Regression เพราะเป็นโมเดลมีความแม่นยำสูงที่สุด

In [9]:
from sklearn.inspection import permutation_importance

importance = permutation_importance(logistic_pipeline, test[feature], test[label], random_state=1234)
importance = pd.Series(importance.importances_mean, index=feature)
importance.sort_values(ascending=False)

GenHlth                 0.008034
BMI                     0.002741
HeartDiseaseorAttack    0.000882
DiffWalk                0.000541
HvyAlcoholConsump       0.000222
Stroke                  0.000139
NoDocbcCost            -0.000015
PhysActivity           -0.000019
Smoker                 -0.000023
AnyHealthcare          -0.000053
Veggies                -0.000237
MentHlth               -0.000304
Income                 -0.000405
PhysHlth               -0.000522
Education              -0.000638
Fruits                 -0.000848
CholCheck              -0.001006
Sex                    -0.001487
HighChol               -0.003792
Age                    -0.005035
HighBP                 -0.008849
dtype: float64

โดยเลือก 5 ฟีเจอร์แรกที่มีค่าความสำคัญสูงสุดมาใช้งาน

In [10]:
features = ["GenHlth", "BMI", "HeartDiseaseorAttack", "DiffWalk", "HvyAlcoholConsump"]

### เฉ​ลย (label) ที่ต้​องกา​รทำนา​ยคื​ออ​ะไร



In [11]:
label = ["ไม่เป็นโรคเบาหวาน", "เป็นโรคเบาหวาน"]

for i in diabetes.Diabetes_binary.unique().astype(int):
    print(f"{i} = {label[i]}")

0 = ไม่เป็นโรคเบาหวาน
1 = เป็นโรคเบาหวาน


### ระบบของตัวเองสาม​ารถใช้ al​gorith​m อะไรได้บ้าง เพ​ราะเห​ตุใด

เกณฑ์ที่กลุ่มเราใช้ในการเลือกใช้โมเดลจะอิงจาก Cheat Sheet ของ Scikit-Learn ดังภาพต่อไปนี้:

<img src="https://scikit-learn.org/1.3/_static/ml_map.png" alt="sklearn-cheat-sheet" width="700">

เนื่องจากชุดข้อมูลที่กลุ่มเรานำมาใช้นั้นเป็นแบบ Classification หรือแบ่งแยกประเภท และมีเป็นข้อมูลที่มี Labeled จากรูปจะมี `SGD Classifier` และ `Kernel Approximation` ที่สามารถนำมาใช้กับข้อมูลของกลุ่มเราได้ และยังมี Algorithm อื่นๆอีกที่สามารถนำมาใช้งานกับข้อมูลของกลุ่มเราได้แต่ไม่ได้่อยู่ใน Cheat Sheet นี้เช่น `Logistic Regression` และ `Decision Tree` เป็นต้น

### ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)
- นายรักษิต รุ่งรัตนไชย ทำหน้าที่เขียนในหัวข้อ 1 2 3 4
- นายบวรรัตน์ ตั้งนรารัชชกิจ ทำหน้าที่เขียนในหัวข้อ 5 6 7
- นายไชยวัตน์ หนูวัฒนา 

### กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)
ในการทำงานครั้งนี้ ไม่ได้ใช้ปัญญาประดิษฐ์ในการทำงานแต่อย่างใด